In [2]:
# import packages
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import KFold
from pandas import ExcelWriter    
from keras import Sequential
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense
from keras.layers import Dropout
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop, Adagrad,Adadelta, RMSprop,Adam
from sklearn.model_selection import StratifiedKFold 
import shap
import re
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests

seed = 100     # random seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
# select the CN subjects to create semi-simulated data
data = pd.read_csv('DATA.csv')   # path of your dataset

CN_data = data[data['DIAGNOSIS']=='CN']
print('No of cognitively normal participants (CN)', len(CN_data))

# arrange the columns
CN_data = CN_data[['ROI1', 'ROI2',...,'ROI145', 'AGE', 'Subject ID']]

# normalize the data
CN_data['Sum'] = CN_data.iloc[:, :-2].sum(axis=1)
numeric_columns = CN_data.columns[:-3]
CN_data[numeric_columns] = CN_data[numeric_columns].div(CN_data['Sum'], axis=0)

In [ ]:
# calculate the Spearman's correlations with the corresponding p-values between the ROIs and AGE using the CN_data,
# and select the top 10 ROIs with highest negative correlation to perturb. 
# Make sure that these ROIs are either white matter or gray matter. These perturbations represents the psuedo-patient patterns.

corr_list = []
p_val_list = []

for column in CN_data.columns[:-3]:  # consider only the ROI columns
    corr, p_val = spearmanr(CN_data['AGE'], CN_data[column], nan_policy='omit')
    corr_list.append(corr)
    p_val_list.append(p_val)

# Adjust p-values for multiple testing
fdr_corr_p_val = multipletests(p_val_list, method='fdr_bh')[1]

# Create DataFrame with correlation results
correlation_results = pd.DataFrame({
    'Column': CN_data.columns[:-3],
    'Correlation': corr_list,
    'p-value': p_val_list,
    'FDR Corrected p-value': fdr_corr_p_val
})
correlation_results = correlation_results.sort_values(by='Correlation', ascending=False)

# select the top 10 columns with highest negative correlations with AGE, along with AGE column, to perturb the data
columns_to_change = correlation_results['Column'].iloc[-10:].tolist() + ['AGE']
print(columns_to_change)

In [ ]:
# introduce perturbations to the selected columns in the CN data to get semi-simulated dataset

def strip_prefix(column_name):
    return column_name.split('_')[-1]

sns.set_style("whitegrid", {'axes.grid': False, 'axes.linewidth': 2})

corr_results = pd.ExcelWriter('path to correlation results file after perturbations.xlsx')
perburbations_results = pd.ExcelWriter('path to file containing the ROI and AGE perturbations results.xlsx')

# perturbing 40%-70% subjects with an increment of 5%. Select the percentage yeilding highest performance at the end. 

percentages = np.arange(0.40, 0.70, 0.05)
for percentage in percentages:
    total_rows = len(CN_data)
    req_rows = int(total_rows * percentage)
    random_indices = np.random.choice(total_rows, req_rows, replace=False)
    df_copy = CN_data.copy().reset_index(drop=True)
    
    random_percentages_vol = np.random.uniform(0.30, 0.50, size=(req_rows,))
    random_percentages_age = np.random.uniform(0.10, 0.30, size=(req_rows,))

    for i, col in enumerate(columns_to_change):
        if col == 'AGE':
            df_copy.iloc[random_indices, df_copy.columns.get_loc(col)] *= 1 + random_percentages_age
            
            # introduce different perturbations to different age ranges
            age_col_index = df_copy.columns.get_loc(col)
            original_ages = df_copy.iloc[random_indices, age_col_index].values
            mask_55_70 = (original_ages > 55) & (original_ages < 70)
            mask_70_77 = (original_ages >= 70) & (original_ages < 77)
            mask_78_85 = (original_ages >= 78) & (original_ages < 85)
            
            indices_55_70 = random_indices[mask_55_70]
            indices_70_77 = random_indices[mask_70_77]
            indices_78_85 = random_indices[mask_78_85]

            df_copy.iloc[indices_55_70, age_col_index] *= 1 + np.random.uniform(0.03, 0.2, size=len(indices_55_70))
            df_copy.iloc[indices_70_77, age_col_index] *= 1 + np.random.uniform(0.05, 0.12, size=len(indices_70_77))
            df_copy.iloc[indices_78_85, age_col_index] *= 1 + np.random.uniform(0.08, 0.1, size=len(indices_78_85))

            mask_97_up = (original_ages >= 97)
            indices_97_up = random_indices[mask_97_up]
            df_copy.iloc[indices_97_up, age_col_index] *= 1 - np.random.uniform(0.01, 0.03, size=len(indices_97_up))
            
            # make sure to keep the age between 55-97 years old
            df_copy.iloc[random_indices, age_col_index] = df_copy.iloc[random_indices, age_col_index].clip(lower=55, upper=97)
        else:
            df_copy.iloc[random_indices, df_copy.columns.get_loc(col)] *= 1 - random_percentages_vol

    corr_list = []
    p_val_list = []

    for column in df_copy.columns[:-3]:
        if df_copy['AGE'].var() != 0 and df_copy[column].var() != 0:  # Check for variation
            corr, p_val = spearmanr(df_copy['AGE'], df_copy[column], nan_policy='omit')
        else:
            corr, p_val = np.nan, np.nan  # Handle the case with no variation
        corr_list.append(corr)
        p_val_list.append(p_val)

    fdr_corr_p_val = multipletests(p_val_list, method='fdr_bh')[1]

    correlation_results_after = pd.DataFrame({
        'Column': df_copy.columns[:-3],
        'Correlation': corr_list,
        'p-value': p_val_list,
        'FDR Corrected p-value': fdr_corr_p_val
    })
    correlation_results_after = correlation_results_after.sort_values(by='Correlation', ascending=True)

    sheet_name = f'Percentage_{round(percentage * 100)}'
    correlation_results_after.to_excel(corr_results, sheet_name=sheet_name, index=False)
    df_copy.to_excel(perburbations_results, sheet_name=f'Data_{round(percentage * 100)}', index=False)

    num_plots = len(columns_to_change) - 1  # Exclude 'AGE'
    num_cols = 5  # Number of columns in the grid
    num_rows = int(np.ceil(num_plots / num_cols))

    fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(100, 20))
    axes = axes.ravel()

    for i, column in enumerate(columns_to_change):
        if column != 'AGE':
            stripped_column = strip_prefix(column)
            ax = axes[i]
            sns.regplot(x='AGE', y=column, data=df_copy, ax=ax, scatter_kws={"marker": "o", "facecolors": "none", 
                                                                            "edgecolors": "b", 's': 500}, line_kws={"linewidth": 10})
            ax.tick_params(labelsize=105)
            ax.set_xlabel("AGE", size=105)
            ax.set_ylabel(f"{stripped_column}", size=105)
            ax.set_title(f'Percentage: {round(percentage * 100)}%', size=65, pad=50)
            ax.spines['bottom'].set_color('black')
            ax.spines['top'].set_color('black')
            ax.spines['right'].set_color('black')
            ax.spines['left'].set_color('black')
            if df_copy['AGE'].var() != 0 and df_copy[column].var() != 0:
                rho, p_value = spearmanr(df_copy['AGE'], df_copy[column], nan_policy='omit')
                ax.text(0.06, 0.009, r'$\rho$ = %.2f, p-value = %.2E' % (rho, p_value),
                        fontsize=70, transform=ax.transAxes, verticalalignment='top')
            else:
                ax.text(0.06, 0.009, 'No variation', fontsize=70, transform=ax.transAxes, verticalalignment='top')

    plt.tight_layout()
    plt.show()
    plt.close(fig)

corr_results.close()
perburbations_results.close()